# **PARTIE A : ANALYSE ET PRÉTRAITEMENT**
## **1. Exploration et reporting** 
* Effectuez une analyse exploratoire des données pour obtenir un premier aperçu. 
* Identifiez les caractéristiques clés et repérez les éventuelles valeurs aberrantes. 
## **2. Prétraitement des données** 
* Normalisez ou standardisez les variables si nécessaire. 
* Vérifiez les données manquantes et appliquez des stratégies de gestion adaptées. 
## **4. Segmentation du marché boursier (Clustering)** 
* Appliquez K-Means avec k=2 et interprétez les groupes obtenus. 
* Déterminez le nombre optimal de clusters via la méthode du coude et silhouette score. 
* Réalisez un clustering hiérarchique et comparez les résultats avec K-Means. 
* Visualisez les clusters sur le plan défini par les deux premières composantes principales. 
## **5. Détection d’anomalies** 
* Identifiez les 5% de jours considérés comme anomalies à l’aide d’un modèle de détection. 
* Expliquez  les  causes  potentielles  de  ces  anomalies  (événements  macroéconomiques,  crises, décisions d’entreprise...). 

**Contexte:**
Cette partie vise à faire une analyse exploratoire des données récoltées et une segmentation du marché boursier

---

# **Prérequis: Collecte des données**

In [11]:
!pip install yfinance

^C


In [ ]:
import yfinance as yf
import pandas as pd

In [ ]:
start_date = "2018-01-01"
end_date = "2024-12-31"

In [ ]:
#téléchargement des données
tickers = ['TSLA', 'AAPL', 'GOOG', 'AMZN', 'META']  
data = yf.download(tickers, start=start_date, end=end_date)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  5 of 5 completed


In [ ]:
data.to_csv("donnees_boursieres.csv")

---

# **1. Analyse et prétraitement des données** 

In [ ]:
# 1. Analyse exploratrice :
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt

In [ ]:
fichier="donnees_boursieres.csv"
df_bours=pd.read_csv(fichier, sep=";")
df_bours.head()

,"Price,Close,Close,Close,Close,Close,High,High,High,High,High,Low,Low,Low,Low,Low,Open,Open,Open,Open,Open,Volume,Volume,Volume,Volume,Volume"
0,"Ticker,AAPL,AMZN,GOOG,META,TSLA,AAPL,AMZN,GOOG..."
1,"Date,,,,,,,,,,,,,,,,,,,,,,,,,"
2,"2018-01-02,40.479835510253906,59.4505004882812..."
3,"2018-01-03,40.4727897644043,60.209999084472656..."
4,"2018-01-04,40.66079330444336,60.47949981689453..."


In [ ]:
#Nettoyage et réorganisation du csv 
df_bours = pd.read_csv("./donnees_boursieres.csv", header=[0, 1])
df_final = df_bours[2:].copy()
df_final.reset_index(drop=True, inplace=True)

#Fusion les colonnes multi-index en colonnes lisibles
df_final.columns = [f"{col[0]}_{col[1]}" if col[0] not in ["Price", "Ticker", "Date"] else f"{col[0]}" for col in df_final.columns]

#Colonne date : renomme
possible_date_col = [col for col in df_final.columns if "Date" in col or "Price" in col]
if possible_date_col:
    df_final.rename(columns={possible_date_col[0]: "Date"}, inplace=True)

#Convertir la colonne Date en datetime
df_final["Date"] = pd.to_datetime(df_final["Date"], errors="coerce")

#Supprimer les lignes sans date
df_final = df_final.dropna(subset=["Date"])
df_final.reset_index(drop=True, inplace=True)
df_final.head()


,Date,Close_AAPL,Close_AMZN,Close_GOOG,Close_META,Close_TSLA,High_AAPL,High_AMZN,High_GOOG,High_META,...,Open_AAPL,Open_AMZN,Open_GOOG,Open_META,Open_TSLA,Volume_AAPL,Volume_AMZN,Volume_GOOG,Volume_META,Volume_TSLA
0,2018-01-03,40.472790,60.209999,53.868855,183.803726,21.150000,41.017975,60.274502,54.058455,183.913211,...,40.543288,59.415001,52.964637,181.026821,21.400000,118071600.0,62176000.0,28604000.0,16886600.0,67822500.0
1,2018-01-04,40.660793,60.479500,54.063931,183.465317,20.974667,40.764191,60.793499,54.420742,185.336503,...,40.545646,60.250000,54.143555,184.032635,20.858000,89738400.0,60442000.0,20092000.0,13880900.0,149194500.0
2,2018-01-05,41.123714,61.457001,54.851696,185.973511,21.105333,41.210661,61.457001,54.952222,186.023264,...,40.757126,60.875500,54.442137,184.719412,21.108000,94640000.0,70894000.0,25582000.0,13574500.0,68868000.0
3,2018-01-08,40.970982,62.343498,55.086086,187.396790,22.427334,41.267071,62.653999,55.301565,188.013876,...,40.970982,61.799999,54.851696,186.321854,21.066668,82271200.0,85590000.0,20952000.0,17994700.0,147891000.0
4,2018-01-09,40.966274,62.634998,55.052250,186.988708,22.246000,41.137818,62.966499,55.266734,187.914354,...,41.017973,62.845001,55.208511,187.814817,22.344000,86336000.0,73226000.0,18050000.0,12393100.0,107199000.0


In [ ]:
df_final.to_csv("donnees_boursieres_nettoyees.csv")